# LangChainの使い方
このノートブックでは、最新の LangChain（v0.3~）を使った各用途別のサンプルを、逐次的に紹介します。

## 1. LLM ラッパー
OpenAI の API 呼び出しを抽象化したモデルラッパーです。

In [ ]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# 生成AIのチャットモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# モデル実行
print(llm.invoke([{"role": "user", "content": "こんにちは"}]).content)

## 2. プロンプトテンプレートとチェイン

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.プロンプト雛型を作成
# input_variablesにパラメータを定義する
# templateに雛型のプロンプトを作成し置換パラメータを埋め込む
prompt = PromptTemplate(
    input_variables=["language", "word"],
    template="次の単語を{language}語に翻訳してください：{word}"
)

# 3.チェインを作成
# 指示(Prompt)、モデル(Model)、回答のパーサー(Parser)を組み合わせたチェイン作成
# StrOutputParserは回答を文字列として取得するパーサー
chain = prompt | llm | StrOutputParser()

# チェインを実行
# 1つ目のチェインの指示に埋め込むパラメータを辞書形式で渡す
print(chain.invoke({
    "language": "日本",
    "word": "りんご"
}))

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.チェインの作成
## 要約プロンプト
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="次の単語について解説してください。: {text}"
)
## 要約チェイン
summary_chain = summary_prompt | llm | StrOutputParser()

## 翻訳プロンプト
translate_prompt = PromptTemplate(
    input_variables=["text"],
    template="英語に翻訳してください。: {text}"
)

translate_chain = translate_prompt | llm | StrOutputParser()

# 3.チェインを組み合わせ
# smmaryの結果をtranslate_chainに渡す
chain = summary_chain | translate_chain

# チェインを実行
# 単語を解説してから英語に翻訳
print(chain.invoke("大規模言語モデル"))


チャット(やり取り)をテンプレート化

In [ ]:
import os
from langchain.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatPromptTemplate
)
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# ------------------------------------------------------
# 単一のチャットをテンプレート化する
# ------------------------------------------------------

# システムプロンプトをテンプレート化
system_template = SystemMessagePromptTemplate.from_template(
    "これから提示するテーマに{language}で答えてください。")
print(system_template.format_messages(language="英語"))
# => [HumanMessage(content="これから提示するテーマに英語で答えてください。")]

# 人間からの単一メッセージをテンプレート化
human_template = HumanMessagePromptTemplate.from_template("本日の{theme}について")
print(human_template.format_messages(theme="天気"))
# => [HumanMessage(content="User: 今日は天気は？")]

# AIの単一メッセージをテンプレート化
ai_template = AIMessagePromptTemplate.from_template(
    "本日の{theme}は{answer}となっています。")
print(ai_template.format_messages(theme="天気", answer="晴れ"))
# => [AIMessage(content='本日の天気は晴れとなっています。')]

# ------------------------------------------------------
# 単一のチャット雛型をまとめる。
# ------------------------------------------------------
print(
    ChatPromptTemplate.from_messages([
        system_template,
        human_template,
        ai_template
    ]).format_prompt(
        language="英語",
        theme="天気",
        answer="晴れ"
    ))

# ------------------------------------------------------
# 初めからまとめて作成する
# ------------------------------------------------------
# 複数のチャットをまとめて定義する
chat_template = ChatPromptTemplate.from_messages([
    ("system", "これから提示するテーマに{language}で答えてください。"),
    ("human", "本日の{theme}について"),
    ("ai", "本日の{theme}は{answer}となっています。")
])

print(
    chat_template.format_prompt(
        language="英語",
        theme="天気",
        answer="晴れ"
    ))

チャットからテキスト生成を行う

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.チャット雛型(あるいは履歴)を作成
chat_template = ChatPromptTemplate.from_messages([
    ("system", "{theme}に関する話題について答えてください。"),
    ("human", "{theme}の{question}は？"),
    ("ai", "{theme}の{question}は{answer}となります。"),
    ("human", "{additional}")
])

# 3.チェインを組み立て
chain = chat_template | llm | StrOutputParser()

# チェインを実行
chain.invoke({
    "theme": "本日",
    "question": "天気",
    "answer": "雨",
    "additional": "傘は必要か？"
})

## 4. エージェント（Agents:LangGraph版）
LangChain版とLangGraph版があるが、LangGraph版への移行が推奨されているためLangGraph版で解説する

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
# from langchain.agents import load_tools
from datetime import datetime,date
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()


# 1.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 2.ツールの設定
# Note:LangChainには便利なツールが標準でいくつか定義されている。
# ただし、ツールによっては外部サービスのAPIキーが必要になるので調べてみてください。
# serpapiは、Google検索をプログラムで実行してくれるAPIサービスのツールですが、APIキーの取得が必要です。
# tools = load_tools(["serpapi"], llm=llm)

# 自作のツールを定義する
# RAGを構築する際は検索する部分をフレームワークを使うか自作する

@tool
def today_is():
    "今日の日付を返す"
    return  datetime.today().date()

@tool
def fetch_weather(dt:date):
    "指定された日付の天気を返す"
    today = datetime.today().date()
    if dt < today:
        return "雨"
    elif dt == today:
        return "曇りのち晴れ"
    else:
        return "晴れ"

tools = [
    today_is,
    fetch_weather
]

# エージェントを構築する
agent = create_react_agent(model=llm, tools=tools, prompt="You are a helpful assistant")
agent.invoke({
    "messages": [{"role": "user", "content": "今日の日付と天気は？"}]
})


## 5. メモリ管理（LangGraph 永続化）

In [ ]:
import os
from typing import Annotated, TypedDict, Union
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from dotenv import load_dotenv

# 環境設定を読込
load_dotenv()

# 1. 会話履歴を State に定義
class State(TypedDict):
    messages: Annotated[list[Union[HumanMessage, AIMessage]], add_messages]

# 2.生成AIモデルを作成
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 3. ノード定義：履歴を渡して応答を得る
def chatbot_node(state: State) -> dict:
    # 最新のメッセージを含む履歴をモデルに渡す
    response: AIMessage = llm.invoke(state["messages"])
    # 応答を履歴に追加して返す
    return {"messages": state["messages"] + [response]}
# 4. グラフのビルド＆コンパイル（MemorySaver を渡す）
builder = StateGraph(State)
builder.add_node("chatbot", chatbot_node)
builder.set_entry_point("chatbot")
memory = MemorySaver()  # in-memory checkpointer
graph = builder.compile(checkpointer=memory)

# 5. 同じ thread_id で実行すると前回までの履歴を保持
config = {"configurable": {"thread_id": "conversation_1"}}

# 最初の発話
initial = [HumanMessage(content="こんにちは、今日の日付は2025年の7月10日です。あなたは誰ですか？")]
result = graph.invoke({"messages": initial}, config)
print(result["messages"][-1].content)  # => AI の応答

# フォローアップ
followup = [HumanMessage(content="今日の日付はいつでしたっけ？")]
result = graph.invoke({"messages": followup}, config)
print(result["messages"][-1].content)  # => 文脈を踏まえた応答

# LangChainの拡張機能を利用した処理
LangChainに関連する機能をサードパーティが作成したものをまとめたもの(Langchain_community)を利用した実装例

In [ ]:
# コミュニティの追加ライブラリを導入
%pip install langchain_community pypdf


## ドキュメントロード & テキスト分割

### 1000文字ごとに分割するが、前後文章の200文字は重複して取得する

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("sample.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)
print(f"チャンク数: {len(chunks)}")

## 埋め込み & ベクターストア
[python_weaviate](12-04_00_00_python_weaviate.ipynb)のサンプルコードを参照

## 検索＋回答（Retrieval Chain）

In [ ]:
import os
import weaviate
from dotenv import load_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate  # または langchain.prompts.prompt
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_weaviate import WeaviateVectorStore

# 1. 接続情報の読込
load_dotenv()

# 2. Weaviate クライアント初期化 (ローカル)
client = weaviate.connect_to_local()

# 3. Embeddings モデルの初期化
embeddings = AzureOpenAIEmbeddings(
    deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_MODEL"),
    api_version=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_API_VERSION")
)

# 4. WeaviateVectorStore のセットアップ
vectorstore = WeaviateVectorStore(
    client=client,
    index_name="Document",       # Weaviate 側のクラス名
    text_key="content",          # プロパティ名 (本文格納フィールド)
    embedding=embeddings,
)

# 5. LLMの初期化
llm = AzureChatOpenAI(
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 6. プロンプトを作成
prompt = PromptTemplate(
    input_variables=["context", "input"],
    template="""
次の文脈をもとに質問に答えてください:
{context}
ユーザ: {input}
アシスタント:""")

# 7. 回答のチェインを定義
combine_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

# 8.回答のチェインを初期化
qa_chain = create_retrieval_chain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=combine_chain,
)

# 9. チェインを実行
result = qa_chain.invoke({"input": "LangChain のメモリ管理方法は？"})
print(result["answer"])

# 10. クライアントのクローズ
client.close()

## コールバック（ストリーミング出力）
生成AIが生成する回答がすべて得られるまで待たず、帰ってきた文字を都度、処理する方式で実装する場合の書き方

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 動作確認用（ストリームで受信したデータを標準出力に表示する）
from langchain.callbacks.manager import CallbackManager

# 1. 接続情報の読込
load_dotenv()

# 2. ストリームハンドラを作成
# ストリーム受信時のコールバック関数をハンドラに設定
streaming_handler = StreamingStdOutCallbackHandler()

# 3. ストリーミング設定でLLMの初期化
llm_stream = AzureChatOpenAI(
    streaming=True,
    callbacks=[streaming_handler],
    azure_deployment=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_MODEL"),
    api_version=os.environ.get("AZURE_OPENAI_LLM_DEPLOYMENT_API_VERSION"),
    temperature=0.7
)

# 4.実行
llm_stream.invoke([{"role":"user","content":"最新ニュースを教えて"}])